In [6]:
import os
import pandas as pd
from ast import literal_eval
from collections import defaultdict
import numpy as np
import json
import pickle
from tqdm import tqdm
tqdm.pandas()

In [2]:
explainability_results = pd.read_csv(
    os.path.join('final_data', 'test_df_interpretability_DEEPLift.csv.gz'), compression='gzip'
)

In [3]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [4]:
explainability_list = flatten(
    explainability_results["interpretability_col"].progress_apply(
        lambda x: literal_eval(x)["interpretability_results"]
    )
)

explainability_list_as_dict = defaultdict(
    lambda: defaultdict(lambda: defaultdict(list))
)

for results_one_excerpt in tqdm(explainability_list):
    for tag, results in results_one_excerpt.items():
        dict_results = dict(results)
        for token, value in dict_results.items():
            explainability_list_as_dict[token][tag]["raw_values"].append(value)

sorted_explainability = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
explainability_tmp = defaultdict(lambda: defaultdict())
final_explainability = defaultdict()

overall_sum = []

for token, results in tqdm(explainability_list_as_dict.items()):
    overall_sum_one_token = 0
    sum_tags = []
    for tag, values in results.items():
        values_list = values["raw_values"]

        values_dict = {
            "mean": np.mean(values_list),
            "median": np.median(values_list),
            "std": np.std(values_list),
            "min": np.min(values_list),
            "max": np.max(values_list),
        }

        if np.any(np.array(list(values_dict.values()))>0.2):
            
            sorted_explainability[token][tag]['mean'] = values_dict['mean']
            sorted_explainability[token][tag]['median'] = values_dict['median']
            sorted_explainability[token][tag]['std'] = values_dict['std']
            sorted_explainability[token][tag]['min'] = values_dict['min']
            sorted_explainability[token][tag]['max'] = values_dict['max']

            sum_values = np.sum(
                np.abs(np.array(list(
                    sorted_explainability[token][tag].values()
                )))
            )

            sum_tags.append(sum_values)

            overall_sum_one_token += sum_values

    tags = list(sorted_explainability[token].keys())
    values_one_tag = list(sorted_explainability[token].values())

    if len(sum_tags)>0:
        for id in np.argsort(sum_tags)[::-1]:
            explainability_tmp[token][tags[id]] = values_one_tag[id]

        overall_sum.append(overall_sum_one_token)

tokens = list(explainability_tmp.keys())
values_one_token = list(explainability_tmp.values())
for id in np.argsort(overall_sum)[::-1]:
    final_explainability[tokens[id]] = values_one_token[id]


100%|██████████| 18046/18046 [01:42<00:00, 175.94it/s]


In [7]:
all_kwords_file = open("results/all_kw_analysis.pkl", "wb")
pickle.dump(final_explainability, all_kwords_file)


## begin here if file is ready

In [1]:
import pickle

In [5]:
a_file = open("results/all_kw_analysis.pkl", "rb")
kwords = pickle.load(a_file)

In [6]:
tags = defaultdict(lambda: defaultdict())

kwords.keys()#['▁2020']['first_level_tags->pillars_2d->Capacities & Response']

dict_keys(['▁[', '▁Colombia', '2020', '▁2020', '▁2021', '▁Bogotá', '▁Venezuela', '▁Syria', ':', '▁•', '▁migrant', '▁(', '▁children', '▁affected', '▁access', '▁2', 'riño', ']', '▁health', '▁violence', '▁impact', '▁2019', '▁humanitar', '▁Rohingya', '▁schools', '▁Bangladesh', '▁Honduras', '▁Nigeria', '▁food', '▁reported', '▁security', 'security', '▁2018', '▁risk', '▁protection', 'kina', '▁hospital', '▁Damasc', '▁in', ',', '▁flood', '▁crisis', '▁education', '▁killed', '▁people', '▁)', '▁3', '▁attack', '▁"', '▁measure', '▁Santander', '▁due', '▁Access', '▁aid', '▁refuge', '▁-', '▁virus', '▁UN', 'ran', '▁population', '▁cases', '▁mobil', '▁Congo', 'no', '▁Bazar', '▁Sudan', '▁barrier', '▁lack', '▁actor', '▁concern', '▁million', '▁Health', '▁Niger', '▁information', '▁school', '▁and', '▁pande', '▁challenges', '▁market', 'hel', '▁1', 'break', '▁discrimina', '▁challenge', '▁border', '▁military', '▁communities', '▁restriction', '▁emergency', 'qqa', '▁the', '▁die', '▁women', '▁need', '▁UNICEF', '▁Sal